In [0]:
CREATE OR REFRESH STREAMING TABLE transaction_user_drugs_silver
(CONSTRAINT valid_timestamp EXPECT (drug_date_taken > cast(DATE'2024-09-01' AS TIMESTAMP)) ON VIOLATION DROP ROW)
COMMENT "Filtered transaction on drugs data"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  u.id AS user_id,
  u.first_name AS user_first_name,
  u.last_name AS user_last_name,
  u.email,
  u.gender,
  u.state_country,
  u.country,
  d.hospital_drug_id,
  d.hospital_npi,
  d.hospital_name,
  d.hospital_city,
  d.drug_name,
  d.drug_brand,
  d.date_taken AS drug_date_taken,
  t.transaction_qty,
  t.transaction_date
FROM 
 (SELECT *
  FROM STREAM(LIVE.users_bronze_raw)) AS u
INNER JOIN STREAM(LIVE.hospital_drug_bronze_raw) d ON u.id = d.user_id
INNER JOIN STREAM(LIVE.transaction_bronze_raw) t ON u.id = t.user_id AND d.hospital_drug_id 
--   (SELECT *
--   FROM STREAM(users_bronze_raw)) AS u
-- INNER JOIN STREAM(hospital_drug_bronze_raw) d ON u.user_id = d.drug_user_id
WHERE
  u.email IS NOT NULL
  AND u.state_country IS NOT NULL
  AND u.country IS NOT NULL
  AND d.drug_brand IS NOT NULL
  AND d.drug_name IS NOT NULL
  AND d.date_taken IS NOT NULL